In [1]:
#imports
import numpy as np
import os
import pandas as pd

import math

from datetime import datetime, timedelta
 
import yfinance as yf

import hvplot.pandas

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import panel as pn
pn.config.theme = 'dark'

In [2]:
source_df = None

In [3]:
symbol = 'SPY'
file_name = f"{symbol}.csv"
source_df = None

In [4]:
if os.path.exists(file_name):
    source_df = pd.read_csv(file_name)

In [5]:

if source_df is None:
    display("download")
    ticker = yf.Ticker(symbol)
    start = datetime.utcnow() - timedelta(days=10*365)
    end=datetime.utcnow() - timedelta(days=1)
    source_df = ticker.history(start=start, end=end)
    source_df.to_csv(f"{symbol}.csv")

In [6]:
display(source_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
2511,2024-02-22 00:00:00-05:00,504.010010,508.489990,503.019989,507.500000,76402500,0.0,0.0,0.0
2512,2024-02-23 00:00:00-05:00,509.269989,510.130005,507.100006,507.850006,61284200,0.0,0.0,0.0
2513,2024-02-26 00:00:00-05:00,508.299988,508.750000,505.859985,505.989990,50386700,0.0,0.0,0.0
2514,2024-02-27 00:00:00-05:00,506.700012,507.160004,504.750000,506.929993,48854500,0.0,0.0,0.0
2515,2024-02-28 00:00:00-05:00,505.329987,506.855011,504.959991,506.260010,50547120,0.0,0.0,0.0


In [7]:
bars_df = source_df.copy()
bars_df["Next Close"] = bars_df["Close"].shift(-1)
bars_df["Next High"] = bars_df["High"].shift(-1)
bars_df["Next Low"] = bars_df["Low"].shift(-1)

#15 and 60 minute window for high,low and open
period = 1
windows = [3, 12]

for window in windows:
    wp = window*period
    bars_df[f"High - {wp}"] = bars_df["High"].rolling(window=window).max()
    bars_df[f"Low - {wp}"] = bars_df["Low"].rolling(window=window).min()
    bars_df[f"Open - {wp}"] = bars_df["Open"].shift(periods=window-1)
    
bars_df = bars_df.dropna()
display(bars_df)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,Next High,Next Low,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12
11,2014-03-18 00:00:00-04:00,155.832355,156.833900,155.665421,156.625244,101804600,0.000,0.0,0.0,155.790619,156.858935,154.797416,156.833900,153.937766,154.279964,157.710268,153.361861,154.113016
12,2014-03-19 00:00:00-04:00,156.641925,156.858935,154.797416,155.790619,176267300,0.000,0.0,0.0,156.700348,156.817195,155.172987,156.858935,154.797416,154.897588,157.710268,153.937766,155.899082
13,2014-03-20 00:00:00-04:00,155.448414,156.817195,155.172987,156.700348,117241000,0.000,0.0,0.0,156.092575,158.456604,155.950065,156.858935,154.797416,155.832355,157.710268,153.937766,156.692006
14,2014-03-21 00:00:00-04:00,157.358425,158.456604,155.950065,156.092575,163128000,0.825,0.0,0.0,155.447083,156.821917,154.768057,158.456604,154.797416,156.641925,158.456604,153.937766,157.084316
15,2014-03-24 00:00:00-04:00,156.629097,156.821917,154.768057,155.447083,121411000,0.000,0.0,0.0,156.184784,156.712921,155.312951,158.456604,154.768057,155.448414,158.456604,153.937766,157.710268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,2024-02-21 00:00:00-05:00,495.420013,497.369995,493.559998,497.209991,59603800,0.000,0.0,0.0,507.500000,508.489990,503.019989,502.869995,493.559998,501.700012,503.500000,490.230011,493.700012
2511,2024-02-22 00:00:00-05:00,504.010010,508.489990,503.019989,507.500000,76402500,0.000,0.0,0.0,507.850006,510.130005,507.100006,508.489990,493.559998,497.720001,508.489990,490.720001,493.519989
2512,2024-02-23 00:00:00-05:00,509.269989,510.130005,507.100006,507.850006,61284200,0.000,0.0,0.0,505.989990,508.750000,505.859985,510.130005,493.559998,495.420013,510.130005,490.720001,496.290009
2513,2024-02-26 00:00:00-05:00,508.299988,508.750000,505.859985,505.989990,50386700,0.000,0.0,0.0,506.929993,507.160004,504.750000,510.130005,503.019989,504.010010,510.130005,490.720001,498.100006


In [8]:
df = bars_df.reset_index()
ys = {
    "Predicted High": df["Next High"], 
    "Predicted Low": df["Next Low"], 
    "Predicted Close": df["Next Close"],
}

X = df.drop(columns=["Next Close", "Next Low", "Next High", "Date"])
display(X)

,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12
0,11,155.832355,156.833900,155.665421,156.625244,101804600,0.000,0.0,0.0,156.833900,153.937766,154.279964,157.710268,153.361861,154.113016
1,12,156.641925,156.858935,154.797416,155.790619,176267300,0.000,0.0,0.0,156.858935,154.797416,154.897588,157.710268,153.937766,155.899082
2,13,155.448414,156.817195,155.172987,156.700348,117241000,0.000,0.0,0.0,156.858935,154.797416,155.832355,157.710268,153.937766,156.692006
3,14,157.358425,158.456604,155.950065,156.092575,163128000,0.825,0.0,0.0,158.456604,154.797416,156.641925,158.456604,153.937766,157.084316
4,15,156.629097,156.821917,154.768057,155.447083,121411000,0.000,0.0,0.0,158.456604,154.768057,155.448414,158.456604,153.937766,157.710268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,2510,495.420013,497.369995,493.559998,497.209991,59603800,0.000,0.0,0.0,502.869995,493.559998,501.700012,503.500000,490.230011,493.700012
2500,2511,504.010010,508.489990,503.019989,507.500000,76402500,0.000,0.0,0.0,508.489990,493.559998,497.720001,508.489990,490.720001,493.519989
2501,2512,509.269989,510.130005,507.100006,507.850006,61284200,0.000,0.0,0.0,510.130005,493.559998,495.420013,510.130005,490.720001,496.290009
2502,2513,508.299988,508.750000,505.859985,505.989990,50386700,0.000,0.0,0.0,510.130005,503.019989,504.010010,510.130005,490.720001,498.100006


In [9]:
for name, y in ys.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    model = ElasticNet()
    model.fit(X_train, y_train)
    predicted_y_values = model.predict(X)
    # create copy of original data
    bars_df[name] = predicted_y_values
    bars_df

/Users/devin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.302e+03, tolerance: 1.881e+03
  model = cd_fast.enet_coordinate_descent(
/Users/devin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.138e+04, tolerance: 1.833e+03
  model = cd_fast.enet_coordinate_descent(
/Users/devin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

In [10]:
display(bars_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Next Low,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12,Predicted High,Predicted Low,Predicted Close
2510,2024-02-21 00:00:00-05:00,495.420013,497.369995,493.559998,497.209991,59603800,0.0,0.0,0.0,507.500000,...,503.019989,502.869995,493.559998,501.700012,503.500000,490.230011,493.700012,497.012659,491.994698,494.757892
2511,2024-02-22 00:00:00-05:00,504.010010,508.489990,503.019989,507.500000,76402500,0.0,0.0,0.0,507.850006,...,507.100006,508.489990,493.559998,497.720001,508.489990,490.720001,493.519989,508.063870,502.969462,505.506403
2512,2024-02-23 00:00:00-05:00,509.269989,510.130005,507.100006,507.850006,61284200,0.0,0.0,0.0,505.989990,...,505.859985,510.130005,493.559998,495.420013,510.130005,490.720001,496.290009,512.128643,507.420555,509.583637
2513,2024-02-26 00:00:00-05:00,508.299988,508.750000,505.859985,505.989990,50386700,0.0,0.0,0.0,506.929993,...,504.750000,510.130005,503.019989,504.010010,510.130005,490.720001,498.100006,509.710466,505.451050,507.547580
2514,2024-02-27 00:00:00-05:00,506.700012,507.160004,504.750000,506.929993,48854500,0.0,0.0,0.0,506.260010,...,504.959991,510.130005,504.750000,509.269989,510.130005,490.720001,498.839996,508.222034,503.797767,506.041867


In [11]:
bars_df["High/Low Success"] = np.where(
    (bars_df["Next Close"] >= bars_df["Predicted Low"]),
    np.where(
        bars_df["Next Close"] <= bars_df["Predicted High"], 1, 0
    ), 0
)

In [12]:
bars_df["Predicted Close Delta"] = bars_df["Predicted Close"] - bars_df["Close"]
bars_df["Next Close Delta"] = bars_df["Next Close"] - bars_df["Close"]
bars_df["Close Product"] = bars_df["Next Close Delta"] * bars_df["Predicted Close Delta"]
bars_df["Close Success"] = np.where(
    bars_df["Close Product"] >= 0, 1, 0
)

In [13]:
actual_close = bars_df.hvplot.line(
    x="Date",
    y="Next Close",
)

In [14]:
predicted_high = bars_df.hvplot.line(
    x="Date",
    y="Predicted High",
)

In [15]:
predicted_low = bars_df.hvplot.line(
    x="Date",
    y="Predicted Low",
)

In [16]:
predicted_close = bars_df.hvplot.line(
    x="Date",
    y="Predicted Close",
)

In [17]:
actual_close * predicted_high * predicted_low * predicted_close

:Overlay
   .Curve.I   :Curve   [Date]   (Next Close)
   .Curve.II  :Curve   [Date]   (Predicted High)
   .Curve.III :Curve   [Date]   (Predicted Low)
   .Curve.IV  :Curve   [Date]   (Predicted Close)

In [18]:
bars_df.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,Next High,...,Low - 12,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success
count,2504.000000,2504.000000,2504.000000,2504.000000,2.504000e+03,2504.000000,2504.0,2504.0,2504.000000,2504.000000,...,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000
mean,285.517912,287.101274,283.817747,285.575733,9.181784e+07,0.020951,0.0,0.0,285.715363,287.241059,...,276.941658,284.001226,287.287815,284.009766,285.777100,0.447284,0.201368,0.139630,0.343982,0.515176
std,98.977724,99.620722,98.319603,99.023397,4.589048e+07,0.167417,0.0,0.0,99.087979,99.683411,...,95.565849,98.300881,99.624356,98.319233,98.977456,0.497313,1.843683,3.332294,10.644030,0.499869
min,152.689025,153.720146,151.993230,152.160889,2.027000e+07,0.000000,0.0,0.0,152.160889,153.720146,...,151.993230,152.689025,152.784365,150.623268,151.850391,0.000000,-12.737797,-27.672806,-116.677875,0.000000
25%,189.933401,190.523531,189.196216,189.901855,6.250578e+07,0.000000,0.0,0.0,189.974274,190.532606,...,186.304076,189.313306,190.864817,189.504793,190.325843,0.000000,-0.563002,-0.994152,-0.675886,0.000000
50%,260.446487,261.593721,258.557049,260.004105,7.966720e+07,0.000000,0.0,0.0,260.110275,261.675825,...,252.987187,258.939769,261.880559,259.245108,260.784609,0.000000,0.153425,0.156372,0.008544,1.000000
75%,385.153791,388.600645,381.982813,385.726250,1.063610e+08,0.000000,0.0,0.0,385.824799,389.290089,...,370.004680,383.563880,388.469990,382.599221,386.000990,1.000000,0.815685,1.502464,0.790518,1.000000
max,509.269989,510.130005,507.100006,507.850006,5.072443e+08,1.906000,0.0,0.0,507.850006,510.130005,...,490.720001,498.839996,512.128643,507.420555,509.583637,1.000000,11.118976,20.159607,129.929973,1.000000


In [19]:
bars_df["Action"] = np.where(
    bars_df["Predicted Close"] > bars_df["Close"], 1, -1
)

In [20]:
display(bars_df.head())
display(bars_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success,Action
11,2014-03-18 00:00:00-04:00,155.832355,156.833900,155.665421,156.625244,101804600,0.000,0.0,0.0,155.790619,...,154.113016,156.415697,155.617700,155.967131,1,-0.658113,-0.834625,0.549278,1,-1
12,2014-03-19 00:00:00-04:00,156.641925,156.858935,154.797416,155.790619,176267300,0.000,0.0,0.0,156.700348,...,155.899082,156.946730,155.455532,156.221643,1,0.431024,0.909729,0.392115,1,1
13,2014-03-20 00:00:00-04:00,155.448414,156.817195,155.172987,156.700348,117241000,0.000,0.0,0.0,156.092575,...,156.692006,156.154694,155.144137,155.624936,1,-1.075412,-0.607773,0.653606,1,-1
14,2014-03-21 00:00:00-04:00,157.358425,158.456604,155.950065,156.092575,163128000,0.825,0.0,0.0,155.447083,...,157.084316,157.742282,156.178394,156.977898,0,0.885323,-0.645493,-0.571469,0,1
15,2014-03-24 00:00:00-04:00,156.629097,156.821917,154.768057,155.447083,121411000,0.000,0.0,0.0,156.184784,...,157.710268,156.484115,155.238153,155.854761,1,0.407678,0.737701,0.300745,1,1


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success,Action
2510,2024-02-21 00:00:00-05:00,495.420013,497.369995,493.559998,497.209991,59603800,0.0,0.0,0.0,507.500000,...,493.700012,497.012659,491.994698,494.757892,0,-2.452100,10.290009,-25.232125,0,-1
2511,2024-02-22 00:00:00-05:00,504.010010,508.489990,503.019989,507.500000,76402500,0.0,0.0,0.0,507.850006,...,493.519989,508.063870,502.969462,505.506403,1,-1.993597,0.350006,-0.697771,0,-1
2512,2024-02-23 00:00:00-05:00,509.269989,510.130005,507.100006,507.850006,61284200,0.0,0.0,0.0,505.989990,...,496.290009,512.128643,507.420555,509.583637,0,1.733631,-1.860016,-3.224582,0,1
2513,2024-02-26 00:00:00-05:00,508.299988,508.750000,505.859985,505.989990,50386700,0.0,0.0,0.0,506.929993,...,498.100006,509.710466,505.451050,507.547580,1,1.557590,0.940002,1.464138,1,1
2514,2024-02-27 00:00:00-05:00,506.700012,507.160004,504.750000,506.929993,48854500,0.0,0.0,0.0,506.260010,...,498.839996,508.222034,503.797767,506.041867,1,-0.888125,-0.669983,0.595029,1,-1


In [21]:
current_position = 0
starting_cash = 1000
current_cash = starting_cash
max_position = 50
for index, row in bars_df.iterrows():
   
    close = row["Close"]
    action = row["Action"]

    if action > 0:
        if action + current_position <= max_position and action * close < current_cash:
            current_position += action
            current_cash -= action*close
        else:
            action = 0
    elif action < 0:
        if action + current_position >= 0:
            current_position += action
            current_cash += -action*close
        else:
            action = 0

    
    bars_df.loc[index, "Position"] = current_position
    bars_df.loc[index, "Cash"] = current_cash

In [22]:
bars_df["Holdings"] = bars_df["Close"] * bars_df["Position"]

# profit
bars_df["Strategy Value"] = bars_df["Holdings"] + bars_df["Cash"]
starting_close = bars_df.iloc[0]["Close"] 
display(f"starting_close {starting_close}")
starting_shares = math.floor(starting_cash / starting_close)
display(f"starting_shares {starting_shares}")
bars_df["Stock Value"] =  bars_df["Close"] * starting_shares

# returns
bars_df["Stock Returns"] = bars_df["Close"].pct_change()
bars_df["Strategy Returns"] = bars_df["Strategy Value"].pct_change()

# cumulative daily returns
bars_df["Stock Cumulative Returns"] = (bars_df["Stock Returns"] + 1).cumprod()
bars_df["Strategy Cumulative Returns"] = (bars_df["Strategy Returns"] + 1).cumprod()

'starting_close 156.625244140625'

'starting_shares 6'

In [23]:
bars_df.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Action,Position,Cash,Holdings,Strategy Value,Stock Value,Stock Returns,Strategy Returns,Stock Cumulative Returns,Strategy Cumulative Returns
2510,2024-02-21 00:00:00-05:00,495.420013,497.369995,493.559998,497.209991,59603800,0.0,0.0,0.0,507.500000,...,-1,0.0,2990.991425,0.000000,2990.991425,2983.259949,0.000906,0.000000,3.174520,2.990991
2511,2024-02-22 00:00:00-05:00,504.010010,508.489990,503.019989,507.500000,76402500,0.0,0.0,0.0,507.850006,...,-1,0.0,2990.991425,0.000000,2990.991425,3045.000000,0.020695,0.000000,3.240218,2.990991
2512,2024-02-23 00:00:00-05:00,509.269989,510.130005,507.100006,507.850006,61284200,0.0,0.0,0.0,505.989990,...,1,1.0,2483.141418,507.850006,2990.991425,3047.100037,0.000690,0.000000,3.242453,2.990991
2513,2024-02-26 00:00:00-05:00,508.299988,508.750000,505.859985,505.989990,50386700,0.0,0.0,0.0,506.929993,...,1,2.0,1977.151428,1011.979980,2989.131409,3035.939941,-0.003663,-0.000622,3.230578,2.989131
2514,2024-02-27 00:00:00-05:00,506.700012,507.160004,504.750000,506.929993,48854500,0.0,0.0,0.0,506.260010,...,-1,1.0,2484.081421,506.929993,2991.011414,3041.579956,0.001858,0.000629,3.236579,2.991011


In [31]:
returns = bars_df.hvplot.line(
    x="Date",
    y=["Strategy Cumulative Returns", "Stock Cumulative Returns"],
).opts(
    title=f"Stock vs Strategy Returns for {symbol} using Elastic Net",
    ylabel="Percent Returns"
)
returns

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [27]:
cash = bars_df.hvplot.line(
    x="Date",
    y="Cash",
)

In [28]:
holdings = bars_df.hvplot.line(
    x="Date",
    y="Holdings",
)

In [29]:
cash*holdings

:Overlay
   .Curve.I  :Curve   [Date]   (Cash)
   .Curve.II :Curve   [Date]   (Holdings)

In [30]:
position = bars_df.hvplot.line(
    x="Date",
    y="Position",
)
position

:Curve   [Date]   (Position)